In [ ]:
!pip install tf2onnx onnxruntime

# Citations

In [ ]:
# @misc{tensorflow2015-whitepaper,
# title={ {TensorFlow}: Large-Scale Machine Learning on Heterogeneous Systems},
# url={https://www.tensorflow.org/},
# note={Software available from tensorflow.org},
# author={
#     Mart\'{\i}n~Abadi and
#     Ashish~Agarwal and
#     Paul~Barham and
#     Eugene~Brevdo and
#     Zhifeng~Chen and
#     Craig~Citro and
#     Greg~S.~Corrado and
#     Andy~Davis and
#     Jeffrey~Dean and
#     Matthieu~Devin and
#     Sanjay~Ghemawat and
#     Ian~Goodfellow and
#     Andrew~Harp and
#     Geoffrey~Irving and
#     Michael~Isard and
#     Yangqing Jia and
#     Rafal~Jozefowicz and
#     Lukasz~Kaiser and
#     Manjunath~Kudlur and
#     Josh~Levenberg and
#     Dandelion~Man\'{e} and
#     Rajat~Monga and
#     Sherry~Moore and
#     Derek~Murray and
#     Chris~Olah and
#     Mike~Schuster and
#     Jonathon~Shlens and
#     Benoit~Steiner and
#     Ilya~Sutskever and
#     Kunal~Talwar and
#     Paul~Tucker and
#     Vincent~Vanhoucke and
#     Vijay~Vasudevan and
#     Fernanda~Vi\'{e}gas and
#     Oriol~Vinyals and
#     Pete~Warden and
#     Martin~Wattenberg and
#     Martin~Wicke and
#     Yuan~Yu and
#     Xiaoqiang~Zheng},
#   year={2015},
# }

# @misc{chollet2015keras,
#   title={Keras},
#   author={Chollet, Fran\c{c}ois and others},
#   year={2015},
#   howpublished={\url{https://keras.io}},
# }

# @article{scikit-learn,
#  title={Scikit-learn: Machine Learning in {P}ython},
#  author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V.
#          and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P.
#          and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and
#          Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},
#  journal={Journal of Machine Learning Research},
#  volume={12},
#  pages={2825--2830},
#  year={2011}
# }

# @inproceedings{Yang2016HierarchicalAN,
#   title={Hierarchical Attention Networks for Document Classification},
#   author={Zichao Yang and Diyi Yang and Chris Dyer and Xiaodong He and Alexander J. Smola and Eduard H. Hovy},
#   booktitle={HLT-NAACL},
#   year={2016}
# }

# @Article{Hunter:2007,
#   Author    = {Hunter, J. D.},
#   Title     = {Matplotlib: A 2D graphics environment},
#   Journal   = {Computing In Science \& Engineering},
#   Volume    = {9},
#   Number    = {3},
#   Pages     = {90--95},
#   abstract  = {Matplotlib is a 2D graphics package used for Python
#   for application development, interactive scripting, and
#   publication-quality image generation across user
#   interfaces and operating systems.},
#   publisher = {IEEE COMPUTER SOC},
#   doi       = {10.1109/MCSE.2007.55},
#   year      = 2007
# }

# Importing Packages and Basic Parameters

In [ ]:
from google.colab import drive
drive.mount('./gdrive', force_remount=True) # 此處需要登入google帳號
# 獲取授權碼之後輸入即可連動雲端硬碟

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import os
import scipy.io as sio
import pickle
import matplotlib.pyplot as plt
import tf2onnx
import onnxruntime as rt
import librosa
import soundfile as sf
# import random as rn

from scipy import stats
from os import listdir
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Average, Concatenate, GRU, Bidirectional, LeakyReLU, Dense, Dropout, Input, Convolution1D, Layer, Add
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers, initializers, constraints
from tensorflow.keras import backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from tensorflow.keras.losses import categorical_crossentropy, mean_squared_error, binary_crossentropy 
from tensorflow.keras.utils import plot_model
from tqdm import tqdm
# import sr_ecg
random_seed = 34
batch_size = 64
num_classes = 1
epochs = 500
#decide the GPUs to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# #target ECG leads to train
# TargetLead = [0,1,2]
#Designate the CPSC2018 data set to load
data_folder_path = './solicited/'
# data_folder_path2 = './TrainingSet2/'
# data_folder_path3 = './TrainingSet3/'

In [ ]:
def keras_2_onnx(model, modelName):
    out_path = modelName + ".onnx"
    spec = (tf.TensorSpec((None, 22050, 1), tf.float32, name="input"),)
    spec2 = (tf.TensorSpec((None, 27), tf.float32, name="input2"),)
    m_proto, _ = tf2onnx.convert.from_keras(model, input_signature=[spec, spec2], output_path=out_path)
    
    out_name = [n.name for n in m_proto.graph.output]
    return out_name

def get_prediction(val_X, modelName):
    model_inference = rt.InferenceSession(modelName + ".onnx")
    
    input_name = model_inference.get_inputs()[0].name
    input_name2 = model_inference.get_inputs()[1].name
    label_name = model_inference.get_outputs()[0].name
    label_name2 = model_inference.get_outputs()[1].name
    onnx_pred = model_inference.run([label_name, label_name2], {input_name: val_X[0].astype(np.float32), input_name2: val_X[1].astype(np.float32)})
    # pred = np.argmax(onnx_pred[0], axis=1)
    return onnx_pred

def get_prediction2(val_X, modelName):
    model_inference = rt.InferenceSession(modelName + ".onnx")
    
    input_name = model_inference.get_inputs()[0].name
    input_name2 = model_inference.get_inputs()[1].name
    onnx_pred = model_inference.run(['output'], {input_name: val_X[0].astype(np.float32), input_name2: val_X[1].astype(np.float32)})
    # pred = np.argmax(onnx_pred[0], axis=1)
    return onnx_pred[0]

# CPU and GPU Checking

In [ ]:
tf.test.is_gpu_available()

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
!cp "./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/solicited.zip" '.'
!unzip solicited.zip -d ./ 
!rm solicited.zip

In [ ]:
!ls

In [ ]:
dataList = listdir(data_folder_path)
# dataList2 = listdir(data_folder_path2)
# dataList3 = listdir(data_folder_path3)

# # dataList, dataList2, dataList3
len(dataList)

# MetforNetEF Data Combining

In [ ]:
# data_folder_path = './gdrive/MyDrive/ColabNotebooks/Competitions/ECGEF/ecg/'
TrainTable1 = pd.read_csv('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODA_TB_Clinical_Meta_Info.csv')
TrainTable2 = pd.read_csv('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODA_TB_Solicited_Meta_Info.csv')
TrainTable = pd.merge(TrainTable2, TrainTable1, on="participant")
# dataXList = TrainTable['filename'].values
# dataYList = TrainTable['tb_status'].values
# TrainPID = TrainTable['participant'].copy()
# TrainPID.unique().shape

In [ ]:
TrainTable.columns

In [ ]:
numerical = TrainTable[['filename', 'participant', 'tb_status','sound_prediction_score', 'age', 'height', 'weight', 'reported_cough_dur', 'heart_rate',
       'temperature']].copy()
categorical = TrainTable[['sex', 'tb_prior', 'tb_prior_Pul',
       'tb_prior_Extrapul', 'tb_prior_Unknown', 'hemoptysis', 'weight_loss', 'smoke_lweek', 'fever', 'night_sweats', 'participant']].copy()
categorical = pd.get_dummies(categorical).copy()
categorical['filename'] = TrainTable['filename']

In [ ]:
# TrainTable = pd.concat([numerical,categorical], axis=1, ignore_index=True)
TrainTable = pd.merge(numerical, categorical, on="filename")
TrainTable.head()

In [ ]:
TrainTable.values[:,3:-1082].shape

In [ ]:
np.unique(TrainTable.values[:,-1082:].sum(axis = 0))

In [ ]:
AllCol = TrainTable.columns[3:]
np.save('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/AllCol',AllCol)
AllCol = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/AllCol.npy', allow_pickle=True)
AllCol

In [ ]:
AllCol[:-1082].shape

In [ ]:
dataXList = TrainTable['filename'].values.copy()
dataYList = TrainTable['tb_status'].values.copy()
TrainPID = numerical['participant'].copy()
TrainPID.unique().shape

In [ ]:
dataXList.shape, dataYList.shape

In [ ]:
# import librosa
import soundfile as sf
maxLength = 0
# maxLength2 = 0
for i in tqdm(dataXList):
    ref, rate = sf.read(data_folder_path+i)
    # mel = librosa.feature.melspectrogram(ref, rate, n_fft=1024, hop_length=4, n_mels=32)
    # refLength2 = mel.shape[1]
    refLength = len(ref)

    if maxLength <= refLength:
      maxLength = refLength
print(maxLength, rate)
    # if maxLength2 <= refLength2:
    #   maxLength2 = refLength2
    #   print(maxLength2, rate)

In [ ]:
# maxLength, maxLength2, mel.shape

In [ ]:
X_list = []
# X_list2 = []
Y_list = []
for i in tqdm(range(len(dataXList))):
    patient = np.zeros((22050,1), dtype=np.float32)
    # patient2 = np.zeros((32,5513), dtype=np.float32)
    rawValues, rate = sf.read(data_folder_path+dataXList[i])
    # print(np.min(rawValues), np.max(rawValues))
    mu = np.nanmean(rawValues)
    std = np.nanstd(rawValues)
    patient[-len(rawValues):,0] = (rawValues-mu)/std
    # rawValues2 = librosa.feature.melspectrogram(rawValues, rate, n_fft=1024, hop_length=4, n_mels=32)
    # patient2[:,-rawValues2.shape[1]:] = rawValues2
    # patient[-len(rawValues):,0] = (rawValues)/std

    X_list.append(patient)
    # X_list2.append(patient2.T)
    Y_list.append(dataYList[i])
X_list = np.asarray(X_list)
# X_list2 = np.asarray(X_list2)
Y_list = np.asarray(Y_list)
print(X_list.shape, Y_list.shape)
# print(X_list.shape, X_list2.shape, Y_list.shape)

In [ ]:
np.save('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAX_list',X_list)
# np.save('./CODAX_list2',X_list2)
np.save('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAY_list',Y_list)

In [ ]:
!ls

In [ ]:
X_list_label = TrainTable.copy()
X_list_label.values[:,3:-1082].shape

In [ ]:
mus = []
stds = []
for i in range(7):
  mu = np.nanmean(X_list_label[AllCol[i]].astype('float32'))
  std = np.nanstd(X_list_label[AllCol[i]].astype('float32'))
  X_list_label[AllCol[i]] = (X_list_label[AllCol[i]].astype('float32')-mu)/std
  X_list_label[AllCol[i]] = X_list_label[AllCol[i]].fillna(0).copy()
  mus.append(mu)
  stds.append(std)
CODAMuStd = np.asarray([mus,stds])
CODAMuStd

In [ ]:
np.save('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAMuStd',CODAMuStd)

In [ ]:
mus = []
stds = []
for i in range(6):
  mu = np.nanmean(X_list_label[AllCol[i]].astype('float32'))
  std = np.nanstd(X_list_label[AllCol[i]].astype('float32'))
  X_list_label[AllCol[i]] = (X_list_label[AllCol[i]].astype('float32')-mu)/std
  X_list_label[AllCol[i]] = X_list_label[AllCol[i]].fillna(0).copy()
  mus.append(mu)
  stds.append(std)
CODAMuStd = np.asarray([mus,stds])
np.save('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAMuStd',CODAMuStd)
CODAMuStd

In [ ]:
CODAMuStd = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAMuStd.npy', allow_pickle=True).astype('float32')
CODAMuStd

In [ ]:
np.save('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAX_list_label',X_list_label.values[:,3:])

In [ ]:
X_list_label.values[:,3:-1082].shape

# MetforNetEF Data Preprocessing

In [ ]:
X_list = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAX_list.npy',allow_pickle=True).astype('float32')
# X_list = np.load('./CODAX_list2.npy',allow_pickle=True).astype('float32')
Y_list = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAY_list.npy',allow_pickle=True).astype('float32')
X_list_label = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/CODAX_list_label.npy',allow_pickle=True).astype('float32')[:,:-1082]
# X_list_test = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/ECGEFX_list_test.npy',allow_pickle=True)
# X_list_label_test = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/ECGEFX_list_label_test.npy',allow_pickle=True)

In [ ]:
X_list_label.shape

In [ ]:
data = Y_list
count, bins_count = np.histogram(data, bins=1000)
pdf = count / sum(count)
cdf = np.cumsum(pdf)
plt.plot(bins_count[1:], pdf, label="PDF")
plt.legend()
# plt.ylim([-0.01, 0.08])
# plt.xlim([-0.01, 1.0])
plt.show()

In [ ]:
plt.plot(X_list[0,:,0])
# plt.ylim([-6.0, 1.0])
plt.show()

# MetforNetEF Splitting Train-Validation Data

In [ ]:
X = X_list
y = Y_list
skf = StratifiedKFold(n_splits=10, random_state = random_seed, shuffle=True)
skf.get_n_splits(X, y)
print(skf)  
FoldIndicesList = []
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    FoldIndicesList.append((train_index,test_index))
FoldIndicesList = np.asarray(FoldIndicesList)
np.save('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/FoldIndicesList',FoldIndicesList)
FoldIndicesList = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/FoldIndicesList.npy',allow_pickle=True)

In [ ]:
FoldIndicesList.shape

# MetforNetEF Architecture

In [ ]:
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs): 
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform') 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer) 
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint) 
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint) 
            self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint) 
        super(AttentionWithContext, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = dot_product(x, self.W) 
        if self.bias:
            uit += self.b 
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u) 
        a = K.exp(ait)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx()) 
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
#model structure
def MetforNet121(length):
  
    main_input = Input(shape=(length,1), dtype='float32', name='main_input')
    x = Convolution1D(12, 3, padding='same')(main_input)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 24, strides = 2, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 24, strides = 2, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 24, strides = 2, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 24, strides = 2, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Convolution1D(12, 48, strides = 2, padding='same')(x)
    x = LeakyReLU(alpha=0.3)(x)
    cnnout = Dropout(0.2)(x)
    x = Bidirectional(GRU(12,return_sequences=True,return_state=False))(cnnout)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    x = AttentionWithContext()(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    main_output = Dense(1,activation='sigmoid')(x)

    return Model(inputs=main_input, outputs=main_output)
model = MetforNet121(5513)
print(model.summary())

In [ ]:
#model structure
def MetforNetEF(length, model):
  
    main_input2 = Input(shape=(27,), dtype='float32', name='main_input2')
    x = model.layers[-2].output
    x = Concatenate(axis = -1)([x, main_input2])
    x = Dense(24,activation=None)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = Dropout(0.2)(x)
    x = Dense(1,activation='tanh')(x)
    main_output = Add()([x, model.layers[-1].output])

    return Model(inputs=[model.input, main_input2], outputs=[main_output, model.layers[-1].output])
model = MetforNetEF(5513,MetforNet121(5513))
print(model.summary())

# MetforNetEF Model Training and 10-fold Cross Validation



In [ ]:
#10-fold training
FoldIndicesList = np.load('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/FoldIndicesList.npy',allow_pickle=True)
lr = 0.001
epochs = 100
for i in range(10):
    model_name = 'CODA_PID_10dense_50folds_sigmoid_fold_double_separate_'+str(i)
    x_train_from_train = X_list[np.setdiff1d(FoldIndicesList[i][0],FoldIndicesList[i-1][1])]
    x_val_from_train = X_list[FoldIndicesList[i-1][1]]
    x_train_from_train_label = X_list_label[np.setdiff1d(FoldIndicesList[i][0],FoldIndicesList[i-1][1])]
    x_val_from_train_label = X_list_label[FoldIndicesList[i-1][1]]
    y_train_from_train = Y_list[np.setdiff1d(FoldIndicesList[i][0],FoldIndicesList[i-1][1])]
    y_val_from_train = Y_list[FoldIndicesList[i-1][1]]
    pre_model = MetforNet121(22050)
    model = MetforNetEF(22050,pre_model)

    opt = keras.optimizers.Adam(lr=lr)
    model.compile(loss=mean_squared_error,
                optimizer=opt,
                metrics=['accuracy'])
    checkpointer = ModelCheckpoint('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/double_separate/'+model_name, verbose=1, save_best_only=True)
    model_history = model.fit( [x_train_from_train, x_train_from_train_label], [y_train_from_train,y_train_from_train],
                            batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpointer],
                            validation_data=([x_val_from_train, x_val_from_train_label], [y_val_from_train,y_val_from_train]))
    with open('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/double_separate/'+model_name+'_history.pickle', 'wb') as file:
        pickle.dump(model.history.history, file)


In [ ]:
AUCScores = []
AUCScores2 = []
for i in tqdm(range(10)):
  pre_model = MetforNet121(22050)
  model = MetforNetEF(22050, pre_model)
  model.load_weights('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/double_separate/CODA_PID_10dense_50folds_sigmoid_fold_double_separate_'+str(i))
  out_name = keras_2_onnx(model, './gdrive/MyDrive/ColabNotebooks/Competitions/CODA/double_separate/CODA_PID_10dense_50folds_sigmoid_fold_double_separate_'+str(i))
  predicted_val, predicted_val2 = get_prediction([X_list[FoldIndicesList[i][1]], X_list_label[FoldIndicesList[i][1]][:,1:]], './gdrive/MyDrive/ColabNotebooks/Competitions/CODA/double_separate/CODA_PID_10dense_50folds_sigmoid_fold_double_separate_'+str(i))
  AUCScores.append(roc_auc_score((Y_list[FoldIndicesList[i][1]]>0.5), predicted_val[:,0]))
  AUCScores2.append(roc_auc_score((Y_list[FoldIndicesList[i][1]]>0.5), predicted_val2[:,0]))


In [ ]:
predicted_val.shape, predicted_val2.shape

In [ ]:
print('AUCScores 1')
print(AUCScores)
print("Median AUC: ", np.median(AUCScores))
print("Mean AUC: ", np.mean(AUCScores))

In [ ]:
print('AUCScores 2')
print(AUCScores2)
print("Median AUC: ", np.median(AUCScores2))
print("Mean AUC: ", np.mean(AUCScores2))

In [ ]:
main_input = Input(shape=(22050,1), dtype='float32', name='main_input')  
main_input2 = Input(shape=(27,), dtype='float32', name='main_input2')
outputs = []
for i in tqdm(range(10)):
  pre_model = MetforNet121(22050)
  model = MetforNetEF(22050, pre_model)
  model.load_weights('./gdrive/MyDrive/ColabNotebooks/Competitions/CODA/double_separate/CODA_PID_10dense_50folds_sigmoid_fold_double_separate_'+str(i))
  model.layers.pop(0)
  newOutputs = model([main_input, main_input2])
  outputs.append(newOutputs)
main_output = Average(name='output')([i[0] for i in outputs])
main_output2 = Average(name='output2')([i[1] for i in outputs])
fin_model = Model(inputs=[main_input, main_input2], outputs=[main_output, main_output2])
print(fin_model.summary())

In [ ]:
out_name = keras_2_onnx(fin_model, './gdrive/MyDrive/ColabNotebooks/Competitions/CODA/double_separate/model')
predicted_val = np.zeros(Y_list.shape)
for i in tqdm(range(predicted_val.shape[0])):
  predicted_temp = get_prediction2([X_list[i:i+1,:], X_list_label[i:i+1,:]], './gdrive/MyDrive/ColabNotebooks/Competitions/CODA/double_separate/model')
  predicted_val[i] += predicted_temp[0,0]

In [ ]:
roc_auc_score((Y_list>0.5), predicted_val)

In [ ]:
predicted_val2 = predicted_val.copy()
predicted_val2[predicted_val2<0] = 0
predicted_val2[predicted_val2>=1] = 1

In [ ]:
roc_auc_score((Y_list>0.5), predicted_val2)

# Inference code

In [ ]:
#!/usr/bin/env python
import onnxruntime as rt
import pandas as pd
import numpy as np
import soundfile as sf


def prediction(input_path, model_path):	
    switch = 1
    AllCol = ['participant', 'filename', 'age', 'height', 'weight',
          'reported_cough_dur', 'heart_rate', 'temperature', 'sex_Female',
          'sex_Male', 'tb_prior_No', 'tb_prior_Not sure', 'tb_prior_Yes',
          'tb_prior_Pul_No', 'tb_prior_Pul_Yes', 'tb_prior_Extrapul_No',
          'tb_prior_Extrapul_Yes', 'tb_prior_Unknown_No',
          'tb_prior_Unknown_Yes', 'hemoptysis_No', 'hemoptysis_Yes',
          'weight_loss_No', 'weight_loss_Yes', 'smoke_lweek_No',
          'smoke_lweek_Yes', 'fever_No', 'fever_Yes', 'night_sweats_No',
          'night_sweats_Yes']
    CODAMuStd = np.asarray([[ 40.817745 , 161.45787  ,  57.327652 ,  46.227486 ,  87.421104 ,
            36.733143 ],
          [ 15.1564245,   8.7601595,  13.59417  ,  51.92165  ,  16.980263 ,
              0.5627685]])
    input_df = pd.read_csv(input_path+'meta_info.csv')[['participant', 'filename']]
    X_list_label = np.zeros((input_df.shape[0], 27))

    try:
        input_df2 = pd.read_csv(input_path+'CODA_TB_Clinical_Meta_Info_Test.csv')
        input_df3 = pd.merge(input_df2, input_df, on="participant")
        numerical = input_df3[['filename', 'participant', 'age', 'height', 'weight', 'reported_cough_dur', 'heart_rate',
            'temperature']].copy()
        categorical = input_df3[['sex', 'tb_prior', 'tb_prior_Pul',
            'tb_prior_Extrapul', 'tb_prior_Unknown', 'hemoptysis', 'weight_loss', 'smoke_lweek', 'fever', 'night_sweats', 'participant']].copy()
        categorical = pd.get_dummies(categorical).copy()
        categorical['filename'] = input_df3['filename']
        input_df3 = pd.merge(numerical, categorical, on="filename")
        temp = input_df3[['participant', 'filename']].copy()
        input_df = temp
        
        switch = 0        
        for i in range(6):
            try:
                input_df3[AllCol[2:][i]] = (input_df3[AllCol[2:][i]].astype('float32')-CODAMuStd[0, i])/CODAMuStd[1, i]
                input_df3[AllCol[2:][i]] = input_df3[AllCol[2:][i]].fillna(0).copy()
                temp = input_df3[AllCol[2:][i]].values
                X_list_label[:, i] += temp
            except:
                X_list_label[:, i] = np.zeros(X_list_label.shape[0])
                
        for i in range(21):
            try:
                temp = input_df3[AllCol[2:][i+6]].values
                X_list_label[:, i+6] += temp
            except:
                X_list_label[:, i+6] = np.zeros(X_list_label.shape[0])
    except:
        input_df = pd.read_csv(input_path+'meta_info.csv')[['participant', 'filename']]

    dataXList = input_df['filename'].values
    X_list = []
    for i in range(len(dataXList)):
        patient = np.zeros((22050,1), dtype=np.float32)
        rawValues, rate = sf.read(input_path+'raw_test_data/'+dataXList[i])
        mu = np.nanmean(rawValues)
        std = np.nanstd(rawValues)
        patient[-len(rawValues):, 0] = (rawValues-mu)/std
        X_list.append(patient)
    X_list = np.asarray(X_list)

    pred = np.zeros(X_list.shape[0])
    model_inference = rt.InferenceSession(model_path+'model.onnx')
    input_name = model_inference.get_inputs()[0].name
    input_name2 = model_inference.get_inputs()[1].name
    label_name = model_inference.get_outputs()[0].name
    label_name2 = model_inference.get_outputs()[1].name    
    for i in range(X_list.shape[0]):
        onnx_pred = model_inference.run([label_name, label_name2], {input_name: X_list[i:i+1, :].astype(np.float32), input_name2: X_list_label[i:i+1, :].astype(np.float32)})
        pred[i] += onnx_pred[switch][0, 0]

    #Paitent id
    df_pred = pd.DataFrame(input_df['participant'], columns=['participant']) 
    df_pred['probability'] = pred
    df_pred_temp = df_pred.dropna().groupby('participant')['probability'].max()
    df_pred_temp_values = df_pred_temp.values
    df_pred_temp_values[df_pred_temp_values>=1] = 1
    df_pred_temp_values[df_pred_temp_values<0] = 0
    df_pred_fin = pd.DataFrame(np.asarray([df_pred_temp.index, df_pred_temp_values]).T, columns=['participant','probability'])
    df_pred_fin.to_csv('./output/predictions.csv', index=False)

    # comfirm that probability of predictions.csv exists
    print(pd.read_csv('./output/predictions.csv')['probability'])
    print('mode :', switch)


if __name__ == '__main__':    
    #holdout sample file path 
    input_path = './input/' # Utilize {'meta_info.csv'  'raw_test_data/*.wav'} according to your preprocessing logic
    # In SC2 you may also load CODA_TB_Clinical_Meta_Info_Test.csv
    # however attempting to load it in SC1 will result in failure
    
    #model path
    model_path = './model/'
    prediction(input_path, model_path)
